In [ ]:
import os
import json
import random
import pandas as pd

In [6]:
parent_dir = "explainer_data"
filename = "explainer_clean_100_1558_15.parquet"
filepath = os.path.join(parent_dir, filename)

# Load data agnostic to parquet / json
if filename.endswith('.parquet'):
    df = pd.read_parquet(filepath)
    data = df.to_dict('records')
    print(f"Loaded {len(data)} items from parquet file {filename}.")
elif filename.endswith('.json'):
    with open(filepath, 'r') as f:
        data = json.load(f)
    print(f"Loaded {len(data)} items from JSON file {filename}.")
else:
    raise ValueError(f"Unsupported file format: {filename}. Please use .parquet or .json files.")

random.shuffle(data)
print(f"Shuffled {len(data)} items.")

Loaded 100 items from parquet file explainer_clean_100_1558_15.parquet.
Shuffled 100 items.


In [7]:
# Randomly sample / print data
sample = random.choice(data)
print(sample)

{'prompt': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a chess grandmaster currently playing a very strong opponent. Assume they will be playing optimally. \n\nShortly you'll be provided with a board state by the user -- please analyze it and think through your possible moves.\n\nYou should refer to moves in UCI notation (e.g., d7d5) and should include your thinking in think tags (e.g., <think> your_thinking </think>) and your answer in answer tags (e.g., <answer> UCI_move </answer>). \n\nAs a technique you may want to consider enumerating possible moves and simulating the likely trajectory that would ensue.<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n8| r . . . . r k .\n7| . b . . n p p p\n6| p . p R p . . .\n5| . . p . P . . .\n4| . . . . . . . .\n3| P . N . . N . .\n2| . P P . . P P P\n1| . . K . . . . R\n   _ _ _ _ _ _ _ _\n   A B C D E F G H\n\n- It is White’s turn to move.\n- No castling rights available.\n- No en passant target square.\n- Halfm